In [1]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset_25' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'HR_data')
output_dir = os.path.join(home, 'video_resources', 'sql_database')

In [2]:
from sql_face.alchemy import get_session
session = get_session(output_dir, db_name)

2023-08-10 18:46:40.097759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import  QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup,Detector, EmbeddingModel,EnfsiImage,EnfsiVideoFrame

Ficheros a importar

In [4]:
database = 'forenface'

emb_file = os.path.join(input_dir,f'embeddings_{database}_hr.npy')
filenames_file = os.path.join(input_dir,f'filenames_{database}_hr.npy')
quality_file = os.path.join(input_dir,f'results_{database}_hr.csv')

emb = np.load(emb_file) 
filename = np.load(filenames_file)
df_quality = pd.read_csv(quality_file)


In [5]:
df_quality

,Path,Quality
0,forenface/063c4b12.png,0.534756
1,forenface/023c3a8.png,0.607371
2,forenface/052rq.png,0.584082
3,forenface/043c4a12.png,0.672223
4,forenface/067lq.png,0.398724
...,...,...
2102,forenface/053rp.png,0.112093
2103,forenface/073c4b12.png,0.348948
2104,forenface/013c5b3.png,0.688999
2105,forenface/052lp.png,0.345345


In [6]:
print(filename.shape,emb.shape, df_quality.shape)


(2107,) (2107, 512) (2107, 2)


In [7]:
df = pd.DataFrame({'Path': filename, 'embedding': emb.tolist()})

In [8]:
df

,Path,embedding
0,forenface/063c4b12.png,"[-2.2092502117156982, 1.4234601259231567, 0.29..."
1,forenface/023c3a8.png,"[-0.3138946294784546, -1.4348164796829224, 1.6..."
2,forenface/052rq.png,"[-0.0037430254742503166, 0.12374058365821838, ..."
3,forenface/043c4a12.png,"[-1.8298367261886597, 0.3146398067474365, -0.2..."
4,forenface/067lq.png,"[0.2452252358198166, 0.1712172031402588, -0.03..."
...,...,...
2102,forenface/053rp.png,"[-0.2834993004798889, -0.44100216031074524, 1...."
2103,forenface/073c4b12.png,"[1.9636752605438232, -0.45613181591033936, -0...."
2104,forenface/013c5b3.png,"[1.3335238695144653, 0.4762285053730011, -0.07..."
2105,forenface/052lp.png,"[0.32279589772224426, 0.6192716360092163, -0.1..."


In [9]:
#solo para forenface
df['im_path'] = df['Path'].str.replace(r'\.png$', '.jpg')
df_quality['im_path'] = df_quality['Path'].str.replace(r'\.png$', '.jpg')
results_data = df.merge(df_quality, on='im_path')

In [ ]:
# para las que no son forenface
df['CI_id'] = df.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))
df_quality['CI_id'] = df_quality.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))
results_data = df.merge(df_quality, on='CI_id')

In [10]:
results_data.shape

(2107, 5)

In [11]:
results_data

,Path_x,embedding,im_path,Path_y,Quality
0,forenface/063c4b12.png,"[-2.2092502117156982, 1.4234601259231567, 0.29...",forenface/063c4b12.jpg,forenface/063c4b12.png,0.534756
1,forenface/023c3a8.png,"[-0.3138946294784546, -1.4348164796829224, 1.6...",forenface/023c3a8.jpg,forenface/023c3a8.png,0.607371
2,forenface/052rq.png,"[-0.0037430254742503166, 0.12374058365821838, ...",forenface/052rq.jpg,forenface/052rq.png,0.584082
3,forenface/043c4a12.png,"[-1.8298367261886597, 0.3146398067474365, -0.2...",forenface/043c4a12.jpg,forenface/043c4a12.png,0.672223
4,forenface/067lq.png,"[0.2452252358198166, 0.1712172031402588, -0.03...",forenface/067lq.jpg,forenface/067lq.png,0.398724
...,...,...,...,...,...
2102,forenface/053rp.png,"[-0.2834993004798889, -0.44100216031074524, 1....",forenface/053rp.jpg,forenface/053rp.png,0.112093
2103,forenface/073c4b12.png,"[1.9636752605438232, -0.45613181591033936, -0....",forenface/073c4b12.jpg,forenface/073c4b12.png,0.348948
2104,forenface/013c5b3.png,"[1.3335238695144653, 0.4762285053730011, -0.07...",forenface/013c5b3.jpg,forenface/013c5b3.png,0.688999
2105,forenface/052lp.png,"[0.32279589772224426, 0.6192716360092163, -0.1...",forenface/052lp.jpg,forenface/052lp.png,0.345345


In [12]:
results_data.drop(['Path_x','Path_y'] , axis=1, inplace = True)

In [13]:
results_data

,embedding,im_path,Quality
0,"[-2.2092502117156982, 1.4234601259231567, 0.29...",forenface/063c4b12.jpg,0.534756
1,"[-0.3138946294784546, -1.4348164796829224, 1.6...",forenface/023c3a8.jpg,0.607371
2,"[-0.0037430254742503166, 0.12374058365821838, ...",forenface/052rq.jpg,0.584082
3,"[-1.8298367261886597, 0.3146398067474365, -0.2...",forenface/043c4a12.jpg,0.672223
4,"[0.2452252358198166, 0.1712172031402588, -0.03...",forenface/067lq.jpg,0.398724
...,...,...,...
2102,"[-0.2834993004798889, -0.44100216031074524, 1....",forenface/053rp.jpg,0.112093
2103,"[1.9636752605438232, -0.45613181591033936, -0....",forenface/073c4b12.jpg,0.348948
2104,"[1.3335238695144653, 0.4762285053730011, -0.07...",forenface/013c5b3.jpg,0.688999
2105,"[0.32279589772224426, 0.6192716360092163, -0.1...",forenface/052lp.jpg,0.345345


# Datos para la base de datos
Revisar estos datos si cambiamos de database. o hacer las queries para obtenerlos. 

In [14]:
detector_id = 2 # mtcnn-serfiq
embeddingModel_id = 6 #El nuevo
qualityModel_id = 1 #ser_fiq

# Creamos/Actualizamos FaceImage

In [16]:
#buscamos la faceimage y la creamos si no existe
limit = 300
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    cI = session.query(CroppedImage).join(Image).filter(Image.path == dato.im_path,CroppedImage.detector_id==detector_id).one_or_none()
    #cI = session.query(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id,CroppedImage.detector_id==detector_id).one_or_none()

    if cI:
        FI = session.query(FaceImage).filter(FaceImage.croppedImages == cI,FaceImage.embeddingModel_id == embeddingModel_id).one_or_none()
        if FI:
            FI.embeddings=dato.embedding

        else:
            new_face_image = FaceImage(croppedImage_id=cI.croppedImage_id,
            embeddingModel_id=embeddingModel_id,
            embeddings=dato.embedding,
            confusion_score=None)
            session.add(new_face_image)
        i-=1
        #session.commit()        

    else:
        print (f'{dato.Path} no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()



2107it [01:06, 31.62it/s]


# Creamos / actualizamos QualityImage

In [17]:
#buscamos la qualityimage y la creamos si no existe
limit = 100
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    FI = session.query(FaceImage).filter(FaceImage.embeddingModel_id == embeddingModel_id) \
                .join(CroppedImage).filter(CroppedImage.detector_id==detector_id) \
                .join(Image).filter(Image.path == dato.im_path).one_or_none()
    #    .filter(CroppedImage.croppedImage_id == dato.CI_id).one_or_none()  (no faceimage)

    if FI:
        QI = session.query(QualityImage).filter(QualityImage.faceImages == FI, QualityImage.qualityModel_id == qualityModel_id).one_or_none()
        if QI:
            QI.quality=dato.Quality

        else:
            new_quality_image = QualityImage(faceImage_id = FI.faceImage_id,
            qualityModel_id = qualityModel_id,
            quality=dato.Quality)
            session.add(new_quality_image)
        i-=1
        #session.commit()        

    else:
        #print (dato.Path + 'no encontrado')
        print (dato.im_path + 'no encontrado')


    if not i:
        i=limit
        session.commit()

session.commit()
session.close()

2107it [00:47, 44.65it/s]
